## Introduction

In this notebook I use PySpark, Keras, and Elephas python libraries to build an end-to-end deep learning pipeline that runs on Spark. Spark is an open-source distributed analytics engine that can process large amounts of data with tremendous speed. PySpark is simply the python API for Spark that allows you to use an easy programming language, like python, and leverage the power of Apache Spark.

My interest in putting together this example was to learn and prototype. More specifically, learn more about PySpark pipelines as well as how I could integrate deep learning into the PySpark pipeline. I ran this entire project using Jupyter on my local machine to build a prototype for an upcoming data science project where the data will be massive. Since I work for IBM, I'll take this entire analytics project (Jupyter Notebook) and move it to IBM. This allows me to do my data ingestion, pipelining, training and deployment on a unified platform and on a much larger Spark cluster. Obviously, if you had a real and sizable project or using image data you would NOT do this on your local machine.

Overall, I found it not too difficult to put together this prototype or working example so I hope others will find it useful.

In [2]:
pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c625d6cd935badb3dd736d66e6de9a02a68d905b163113ba4a541464c6edb025
  Stored in directory: c:\users\surya.dev\appdata\local\pip\cache\wheels\1d\27\68\1382001655ef41217e1dd34d59aa777612135379bab64279e9
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install keras
!pip install tensorflow

In [30]:
pip install elephas==3.0.0 --user



  Created wheel for elephas: filename=elephas-3.0.0-py3-none-any.whl size=26261 sha256=fe6b112e04a9a02841db0c3d0b0fb4c4f10ce25ae1568635ce129f3d2ce56b3b
  Stored in directory: c:\users\surya.dev\appdata\local\pip\cache\wheels\d4\ea\b7\5ad17bc2a7134e08d9715156f1407801c4171f5a14af9ab868
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463482 sha256=523e12812915e666dbaac20cc5b739d2e30d28225af567108a788114aef76511
  Stored in directory: c:\users\surya.dev\appdata\local\pip\cache\wheels\d1\b2\3a\d019c9382e6429910cef8ef0aae6a1970bfbbba13562590d70
Successfully built elephas pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.2.0
    Uninstalling pyspark-3.2.0:
      Successfully uninstalled pyspark-3.2.0
  Attempting uninstall: elephas
    Found existing installation: elephas 3.1.0
    Uninstalling elephas-3.1.0:
      Successfully uninstalled elephas-3.1.0


<div class="alert alert-block alert-info">
<b>Step 1:</b> Import Libraries
</div>

In [6]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

# Keras / Deep Learning
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from keras.optimizers import Adam

# Elephas for Deep Learning on Spark
from elephas.ml_model import ElephasEstimator

ModuleNotFoundError: No module named 'tensorflow.keras'

<div class="alert alert-block alert-info">
<b>Step 2:</b> Start Spark Session
</div>

If you're using a version of Spark < 2.x your Spark session code may be slightly different

In [ ]:
# Spark Session
# conf = SparkConf().setAppName('Spark DL Tabular Pipeline').setMaster('local[6]')
conf = SparkConf().setAppName('Spark DL Tabular Pipeline')
sc = SparkContext('local', conf=conf)
sql_context = SQLContext(sc)

In [ ]:
sc

<div class="alert alert-block alert-info">
<b>Step 3:</b> Load and Preview Data
</div>

Here we'll load the data. The data we'll use comes from a [Kaggle competition](https://www.kaggle.com/janiobachmann/bank-marketing-dataset). It's a typical banking dataset. I use the `inferSchema` parameter here which helps to identify the feature types when loading in the data. Per the [PySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html) this _"requires one extra pass over the data"_. Since the bank data I'm loading only has ~11k observations it doesnt take long at all, but it may be worth noting if you have a very large dataset.

After we load the data we can see the schema and the various feature types. All our features are either `string` type or `integer`. We then preview the first 5 observations. I'm pretty familair with with Pandas python library so through this example you'll see me use `toPandas()` to convert the spark dataframe to a pandas dataframe and do some manipulations. Not right or wrong, just easier for me.

Finally, we'll drop the 2 date columns since we won't be using those in our deep learning model. They could be possibly significant and featurized by I decided to just drop them all together.

In [ ]:
# Load Data to Spark Dataframe
df = sql_context.read.csv('C:/Users/surya.dev/PycharmProjects/pyspark_dl_pipeline-master/pyspark_dl_pipeline-master/bank.csv',
                    header=True,
                    inferSchema=True)

In [ ]:
# View Schema
df.printSchema()

In [ ]:
# Preview Dataframe (Pandas Preview is Cleaner)
df.limit(5).toPandas()

In [ ]:
# Drop Unnessary Features (Day and Month)
df = df.drop('day', 'month')

In [ ]:
# Preview Dataframe
df.limit(5).toPandas()

<div class="alert alert-block alert-info">
<b>Step 4:</b> Create the Spark Data Pipeline
</div>

Now we create the pipeline using PySpark. This essentially takes your data and, per the feature lists you pass, will do the transformations and vectorizing so it is ready for modeling. I referenced the ["Extracting, transforming and selecting features"](https://spark.apache.org/docs/latest/ml-features.html) Apache Spark documentation a lot for this pipeline and project.

Below is a helper function to select from the numeric features which ones to standardize based on the kurtosis or skew of that feature. The current defaults for `upper_skew` and `lower_skew` are just general guidelines (depending where you read), but you can modify the upper and lower skew as desired.

In [ ]:
# Helper function to select features to scale given their skew
def select_features_to_scale(df=df, lower_skew=-2, upper_skew=2, dtypes='int32', drop_cols=['']):
    
    # Empty Selected Feature List for Output
    selected_features = []
    
    # Select Features to Scale based on Inputs ('in32' type, drop 'ID' columns or others, skew bounds)
    feature_list = list(df.toPandas().select_dtypes(include=[dtypes]).columns.drop(drop_cols))
    
    # Loop through 'feature_list' to select features based on Kurtosis / Skew
    for feature in feature_list:

        if df.toPandas()[feature].kurtosis() < -2 or df.toPandas()[feature].kurtosis() > 2:
            
            selected_features.append(feature)
    
    # Return feature list to scale
    return selected_features

Now we'll get into the actual data pipeline. The feature list selection part can be further enhanced to be more dynamic vs listing out each feature, but for this small dataset I just left it as is with `cat_features`, `num_features`, and `label`. Selecting features by type can be done similar to how I did it in the `select_features_to_scale` helper function using something like this `spark_df.toPandas().select_dtypes(include=['object']).columns)` which would return a list of all the columns in your spark dataframe that are object or string type.

The first thing we want to do is create an empty list called `stages`. This will contain each step that the data pipeline needs to to complete all transformations within our pipeline. I print out each step of the stages after the pipeline so you can see the sequential steps from my code to a list.

The second part is going to be a basic loop to go through each categorical feature from our list `cat_features` and then index and encode those features using one-hot encoding. `StringIndexer` encodes your categorical feature to a feature index with the highest frequency label (count) as feature index `0` and so on. I will preview the transformed data frame after the pipeline, Step 5, where you can see each feature index created from the categorical features. For more information and a basic example of StringIndexer check the [here](https://spark.apache.org/docs/latest/ml-features.html#stringindexer)

Within the loop we also do some one-hot encoding (OHE) using the `OneHotEncoderEstimator`. This function only takes a label index so if you have categorical data (objects or strings) you have to use `StringIndexer` so you can pass a label index to the OHE estimator. One nice thing I found from looking at dozens of examples was that you can chain `StringIndexer` output right into the OHE estimator using `string_indexer.getOutputCol()`. If you have a lot of features to transform you'll want to do some thinking about the names, `OutputCol`, because you can't just overwrite feature names so get creative. We'll append all those pipeline steps within our loop into our pipeline list `stages`.

Next we use `StringIndexer` again on our label feature or dependent variable. And then we'll move on to scaling the numeric variables using the `select_features_to_scale` helper function from above. Once that list is selected we'll vectorize those features using `VectorAssembler` and then standardize the features within that vector using `StandardScaler`. Then we append those steps to our ongoing pipeline list `stages`.

The last step is just assembling all our features into a single vector. We'll find the numeric features from the list `num_features` that were not scaled by just using the difference between our `unscaled_features` (the name of the selected numeric feature TO scale) list and the original list of numeric features `num_features`. Then we assemble or vectorize all the categorical OHE features and numeric features and add that step to our pipeline `stages`. And finally, we add in the `scaled_features` to our `assembled_inputs` to get a final and single vector of features for our modeling.

In [ ]:
# Spark Pipeline
cat_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
num_features = ['age','balance','duration','campaign','pdays','previous']
label = 'deposit'

# Pipeline Stages List
stages = []

# Loop for StringIndexer and OHE for Categorical Variables
for features in cat_features:
    
    # Index Categorical Features
    string_indexer = StringIndexer(inputCol=features, outputCol=features + "_index")
    
    # One Hot Encode Categorical Features
    encoder = OneHotEncoder(inputCols=[string_indexer.getOutputCol()],
                                     outputCols=[features + "_class_vec"])
    # Append Pipeline Stages
    stages += [string_indexer, encoder]
    
# Index Label Feature
label_str_index =  StringIndexer(inputCol=label, outputCol="label_index")

# Scale Feature: Select the Features to Scale using helper 'select_features_to_scale' function above and Standardize 
unscaled_features = select_features_to_scale(df=df, lower_skew=-2, upper_skew=2, dtypes='int32', drop_cols=['id'])

unscaled_assembler = VectorAssembler(inputCols=unscaled_features, outputCol="unscaled_features")
scaler = StandardScaler(inputCol="unscaled_features", outputCol="scaled_features")

stages += [unscaled_assembler, scaler]

# Create list of Numeric Features that Are Not Being Scaled
num_unscaled_diff_list = list(set(num_features) - set(unscaled_features))

# Assemble or Concat the Categorical Features and Numeric Features
assembler_inputs = [feature + "_class_vec" for feature in cat_features] + num_unscaled_diff_list

assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="assembled_inputs") 

stages += [label_str_index, assembler]

# Assemble Final Training Data of Scaled, Numeric, and Categorical Engineered Features
assembler_final = VectorAssembler(inputCols=["scaled_features","assembled_inputs"], outputCol="features")

stages += [assembler_final]

We can see all the steps within our pipeline by looking at our `stages` list that we've been sequentially adding.

In [ ]:
stages

<div class="alert alert-block alert-info">
<b>Step 5:</b> Run Data Through the Spark Pipeline
</div>

Now that the _"hard"_ part is over we can simply pipeline the stages and fit our data to the pipeline by using `fit()`. Then we actually transform the data by using `transform`. We can now preview our newly transformed PySpark dataframe with all the original and transformed features.

In [ ]:
# Set Pipeline
pipeline = Pipeline(stages=stages)

# Fit Pipeline to Data
pipeline_model = pipeline.fit(df)

# Transform Data using Fitted Pipeline
df_transform = pipeline_model.transform(df)

In [ ]:
# Preview Newly Transformed Data
df_transform.limit(5).toPandas()

In [ ]:
# Data Structure Type is a PySpark Dataframe
type(df_transform)

<div class="alert alert-block alert-info">
<b>Step 6:</b> Final Data Prep before Deep Learning Model
</div>

There are couple last and quick things we need to do before modeling. First is to create a PySpark dataframe that only contains 2 vectors from the recently transformed dataframe. We only need the: `features` (X) and `label_index` (y) features for modeling. It's easy enough to do with PySpark with the simple `select` statement. Then, just cause, we preview the dataframe.

Finally, we want to shuffle our dataframe and then split the data into train and test sets. You always want to shuffle the data prior to modeling to avoid any bias from how the data may be sorted or otherwise organized and specifically shuffling prior to splitting the data.

In [ ]:
# Select only 'features' and 'label_index' for Final Dataframe
df_transform_fin = df_transform.select('features','label_index')
df_transform_fin.limit(5).toPandas()

In [ ]:
# Shuffle Data
df_transform_fin = df_transform_fin.orderBy(rand())

In [ ]:
# Split Data into Train / Test Sets
train_data, test_data = df_transform_fin.randomSplit([.8, .2],seed=1234)

<div class="alert alert-block alert-info">
<b>Step 7:</b> Build a Deep Learning Model
</div>

We'll now build a basic deep learning model using Keras. Keras is described as: _"a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano."_ in the [Keras documentation](https://keras.io/). I find Keras to be one of the easiest deep learning APIs for python. Also, I found an extension of Keras that allowed me to do easy distributed deep learning on Spark that could integrate with my PySpark pipeline.

First we need to determine the number of classes as well as the number of inputs from our data so we can plug those values into our Keras deep learning model.

In [ ]:
# Number of Classes
nb_classes = train_data.select("label_index").distinct().count()

# Number of Inputs or Input Dimensions
input_dim = len(train_data.select("features").first()[0])

Next we create a basic deep learning model. Using the `model = Sequential()` feature from Keras, it's easy to simply add layers and build a deep learning model the all the desired settings (# of units, dropout %, regularization - l2, activation functions, etc.) I selected the common Adam optimizer and binary cross-entropy since out outcome label is binary.

In [ ]:
# Set up Deep Learning Model / Architecture
model = Sequential()
model.add(Dense(256, input_shape=(input_dim,), activity_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(256, activity_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

Once the model is built we can view the architecture. Notice that we went from 30 inputs/parameters to 74,242. The beauty (sometimes laziness :) ) of deep learning is the automatic feature engineering.

In [ ]:
# Model Summary
model.summary()

<div class="alert alert-block alert-info">
<b>Step 8:</b> Distributed Deep Learning
</div>

Now that we have a model built, using Keras as our deep learning framework, we want to run that model on Spark to leverage its distributed analytic engine. We do that by using a python library and an extension to Keras called [Elephas](https://github.com/maxpumperla/elephas). Elephas makes it pretty easy to run your Keras models on Apache spark with few lines of configuration. I found Elephas to be easier and more stable to use than the several other libraries I read about and tried.

The first thing we do with Elephas is create an estimator similar to some of the PySpark pipeline items above. We can set the optimizer settings right from Keras optimizer function and then pass that to our Elephas estimator. I only explicitly use Adam optimizer with a set learning rate, but you can use any [Keras optimizer](https://keras.io/optimizers/) with their respective parameters (clipnorm, beta_1, beta_2, etc.).

Then within the Elephas estimator you specify a variety of items: features column, label column, # of epochs, batch size for training, validation split of your training data, loss function, metric, etc. I just used the settings from an [Elephas example](https://github.com/maxpumperla/elephas/blob/master/examples/ml_pipeline_otto.py) and modified the code slightly.

Notice that after we run the estimator the output, `ElephasEstimator_31afcd77fffd`, looks similar to one of our pipeline `stages` list items. This can be passed directly into our PySpark pipeline to fit and transform our data which we'll do in the next step!

In [ ]:
# Set and Serialize Optimizer
optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

# Initialize SparkML Estimator and Get Settings
estimator = ElephasEstimator()
estimator.setFeaturesCol("features")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(1)
estimator.set_epochs(25) 
estimator.set_batch_size(64)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

<div class="alert alert-block alert-info">
<b>Step 9:</b> Distributed Deep Learning Pipeline and Results
</div>

Now that are deep learning model is to be run on Spark, using Elephas, we can pipeline line it exactly how we did above using `Pipeline()`. You could append this to our `stages` list and do all of this with one pass with a new dataset now that it's all built out which would be super cool!

I created another helper function below called `dl_pipeline_fit_score_results` that takes the deep learning pipeline `dl_pipeline` and then does all the fitting, transforming, and prediction on both the train and test data sets. It also outputs the accuracy for both data sets and their confusion matrices.

In [ ]:
# Create Deep Learning Pipeline
dl_pipeline = Pipeline(stages=[estimator])

In [ ]:
def dl_pipeline_fit_score_results(dl_pipeline=dl_pipeline,
                                  train_data=train_data,
                                  test_data=test_data,
                                  label='label_index'):
    
    fit_dl_pipeline = dl_pipeline.fit(train_data)
    pred_train = fit_dl_pipeline.transform(train_data)
    pred_test = fit_dl_pipeline.transform(test_data)
    
    pnl_train = pred_train.select(label, "prediction")
    pnl_test = pred_test.select(label, "prediction")
    
    pred_and_label_train = pnl_train.rdd.map(lambda row: (row[label], row['prediction']))
    pred_and_label_test = pnl_test.rdd.map(lambda row: (row[label], row['prediction']))
    
    metrics_train = MulticlassMetrics(pred_and_label_train)
    metrics_test = MulticlassMetrics(pred_and_label_test)
    
    print("Training Data Accuracy: {}".format(round(metrics_train.precision(),4)))
    print("Training Data Confusion Matrix")
    display(pnl_train.crosstab('label_index', 'prediction').toPandas())
    
    print("\nTest Data Accuracy: {}".format(round(metrics_test.precision(),4)))
    print("Test Data Confusion Matrix")
    display(pnl_test.crosstab('label_index', 'prediction').toPandas())

Let's use our new deep learning pipeline and helper function on both data sets and test our results!

In [ ]:
dl_pipeline_fit_score_results(dl_pipeline=dl_pipeline,
                              train_data=train_data,
                              test_data=test_data,
                              label='label_index');

## Conclusion

I hope that this example has been helpful. I know it was for me in learning more about PySpark pipelines and doing deep learning on spark using an easy deep learning framework like Keras. Like I mentioned, I ran all this locally with little to no issue. My main objective was to prototype something for an upcoming project that will contain a massive dataset. Luckily working for IBM allows me to leverage Watson so I'll train and deploy on a large Spark cluster. My hope is that you (and myself) can use this as a template while making few changes to things like the spark session, data, feature selection, and maybe adding or removing some pipeline stages. As always, thanks for reading and good luck on your next project!